# Preparing to play

This notebook has my work towards an implementation of the play function in play.py

Initial imports

In [2]:
import pandas as pd
import numpy as np
from processing import *

Get Jeopardy data and preprocess it

In [3]:
jeopardy_data = pd.read_csv("jeopardy.csv")
preprocess(jeopardy_data)

Ensure each category has 5 or more questions, and pick 7 random categories out of those available.

In [4]:
# Makes it easier to change the variables later
categories_to_show = 7
min_questions = 5

In [5]:
categories = jeopardy_data.category
categories = categories.value_counts().apply( \
    lambda c: True if c >= min_questions else False)
categories = categories.sample(categories_to_show)
categories

RESTAURANTS                        True
CAR TUNES                          True
CELEBRATED ON THEIR BIRTHDAYS      True
STAR TREK JARGON                   True
EUROPEAN ARTISTS                   True
THE LOVELY BONES                   True
A 5-CHAPTER STORY ABOUT STORIES    True
Name: category, dtype: bool

Get 5 questions for the available categories

In [6]:
questions = jeopardy_data[jeopardy_data.category.apply( \
    lambda c: categories[c] if c in categories else False)]
category_question_count = {}
def incr(key):
    category_question_count.setdefault(key, 0)
    category_question_count[key] += 1
    return category_question_count[key] <= 5
questions = questions[questions.category.apply(incr)].reset_index(drop=True)
questions

,show_number,air_date,round,category,value,question,answer,value_float,air_year
0,4300,2003-04-18,Jeopardy!,CAR TUNES,$200,"In 1984 Sammy Hagar sang, ""Write me up a 125, ...",55,200.0,2003
1,4300,2003-04-18,Jeopardy!,CAR TUNES,$400,Prince drove this tiny title auto to No. 6 in ...,"""Little Red Corvette""",400.0,2003
2,4300,2003-04-18,Jeopardy!,CAR TUNES,$600,"This ""Little"" Beach Boys vehicle can ""do a hun...","""Little Deuce Coupe""",600.0,2003
3,4300,2003-04-18,Jeopardy!,CAR TUNES,$800,(Cheryl of the Clue Crew at the Henry Ford Mus...,LaSalle,800.0,2003
4,4300,2003-04-18,Jeopardy!,CAR TUNES,$1000,In 1976 Rose Royce did the dirty work & went t...,"""Car Wash""",1000.0,2003
5,6197,2011-07-19,Jeopardy!,CELEBRATED ON THEIR BIRTHDAYS,$200,"National Inventors Day, February 11",Thomas Edison,200.0,2011
6,6197,2011-07-19,Jeopardy!,CELEBRATED ON THEIR BIRTHDAYS,$400,"International Nurses Day, May 12",(Florence) Nightingale,400.0,2011
7,6197,2011-07-19,Jeopardy!,CELEBRATED ON THEIR BIRTHDAYS,$600,"Arbor Day in California, March 7",Luther Burbank,600.0,2011
8,6197,2011-07-19,Jeopardy!,CELEBRATED ON THEIR BIRTHDAYS,$800,"American Painters Day, February 3",Norman Rockwell,800.0,2011
9,6197,2011-07-19,Jeopardy!,CELEBRATED ON THEIR BIRTHDAYS,$1000,"Peter Pan Day, May 9",J.M. Barrie,1000.0,2011


Print out the categories and money values for each question in the category

In [11]:
# Terminals are typically 80 characters wide
view_width = 80
cell_width, expanded_cells = divmod(view_width, categories_to_show)
cell_width -= 1  # Add space for a pipe character
cell_template_str = "{{:^{}}}|"
cell_format_str = cell_template_str.format(cell_width)

In [12]:
longest_category = max(categories.keys(), key=len)
lines, extra_line = divmod(len(longest_category), cell_width)
if extra_line > 0:
    lines += 1

for line in range(lines):
    extender = expanded_cells
    for col in categories.keys():
        start = line * cell_width
        end = start + cell_width
        print(cell_format_str.format(col[start:end]), sep="", end="")
        if extender > 0:
            print(" ", end="")
            extender -= 1
    print()
print("-" * view_width)  # Separator between categories and monies

RESTAURANT| CAR TUNES | CELEBRATED| STAR TREK |EUROPEAN A|THE LOVELY|A 5-CHAPTE|
    S     |           |  ON THEIR |   JARGON  |  RTISTS  |   BONES  |R STORY AB|
          |           | BIRTHDAYS |           |          |          |OUT STORIE|
          |           |           |           |          |          |    S     |
--------------------------------------------------------------------------------


Print out the money values for each selected question

In [15]:
for row in range(min_questions):
    extender = expanded_cells
    for col in range(categories_to_show):
        money_value = "${}".format((row + 1) * 100)  # Replace with actual value
        print(cell_format_str.format(money_value), end="", sep="")
        if extender > 0:
            print(" ", end="")
            extender -= 1
    print("\n", "-" * view_width, sep="")

   $100   |    $100   |    $100   |    $100   |   $100   |   $100   |   $100   |
--------------------------------------------------------------------------------
   $200   |    $200   |    $200   |    $200   |   $200   |   $200   |   $200   |
--------------------------------------------------------------------------------
   $300   |    $300   |    $300   |    $300   |   $300   |   $300   |   $300   |
--------------------------------------------------------------------------------
   $400   |    $400   |    $400   |    $400   |   $400   |   $400   |   $400   |
--------------------------------------------------------------------------------
   $500   |    $500   |    $500   |    $500   |   $500   |   $500   |   $500   |
--------------------------------------------------------------------------------
